In [59]:
from dotenv import load_dotenv
import os
from groq import Groq
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from langgraph.graph import Graph
from langgraph import Node, LLMNode

ImportError: cannot import name 'Node' from 'langgraph' (unknown location)

Basic Functions

In [37]:
# Load the variables from the .env file into the environment
load_dotenv()

# Retrieve the API keys using os.getenv
groq_api_key = os.getenv("GROQ_API_KEY")

# Optionally, check that the keys are loaded properly
if not groq_api_key:
    raise ValueError("GROQ_API_KEY not found. Please add it to your .env file.")

# Print (or use) the API keys as needed
print("Groq API Key:", groq_api_key)



Groq API Key: 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


In [35]:
def LLM(prompt: str) -> str:
    """Generate a completion using the Groq API."""
    
    completion = groq_client.chat.completions.create(
        messages=[{"role": "system", "content": prompt}],
        model="llama-3.3-70b-versatile",
        temperature=0.3,
        max_tokens=150
    )
    
    return completion.choices[0].message.content

In [36]:
val = LLM("What is the capital of France?")
val

'The capital of France is Paris.'

Initial Information

In [43]:
df = pd.read_csv('Copy of OpenSecrets.org _ FTX_Alameda Research Contributions - Direct Contributions & JFC Distributions.csv')

In [50]:
df['date'] = pd.to_datetime(df['date'])
val2 = df.dtypes
val2

cycle                     int64
State/Federal            object
contribid                object
contrib                  object
City                     object
State                    object
Zip                      object
Fecoccemp                object
orgname                  object
ultorg                   object
date             datetime64[ns]
amount                   object
recipid                  object
recipient                object
party                    object
recipcode                object
type                     object
fectransid               object
pg                       object
cmteid                   object
dtype: object

In [51]:
val = df.columns
val

Index(['cycle', 'State/Federal', 'contribid', 'contrib', 'City', 'State',
       'Zip', 'Fecoccemp', 'orgname', 'ultorg', 'date', 'amount', 'recipid',
       'recipient', 'party', 'recipcode', 'type', 'fectransid', 'pg',
       'cmteid'],
      dtype='object')

Extract Schema from CSV

In [52]:
sample_schema = """
Table Name: employees
Columns:
- id: INTEGER
- name: TEXT
- salary: FLOAT
- department: TEXT
"""
prompt = f"Based on the data, the columns are {val}. The datatypes are {val2} Create a schema for the data. It should look like {sample_schema}"
schema = LLM(prompt)
print(schema)

Based on the provided data, here is a schema for the table:

Table Name: Contributions
Columns:
- cycle: INTEGER
- State_Federal: TEXT
- contribid: TEXT
- contrib: TEXT
- City: TEXT
- State: TEXT
- Zip: TEXT
- Fecoccemp: TEXT
- orgname: TEXT
- ultorg: TEXT
- date: DATE
- amount: DECIMAL(10, 2)
- recipid: TEXT
- recipient: TEXT
- party: TEXT
- recipcode: TEXT
- type: TEXT
- fectransid: TEXT
- pg: TEXT
- cmteid: TEXT

Note:
- I've used DECIMAL


In [ ]:
# Ensure your file is not named "langgraph.py" to avoid shadowing the package.
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

# This function represents your node that uses an LLM to convert a natural language query into SQL.
def generate_sql_node(state):
    # The state already contains a "schema" field that is baked in.
    query = state["query"]
    schema = state["schema"]
    prompt = (
        "You are a SQL expert. Given the following table schema:\n"
        f"{schema}\n"
        "Convert the natural language query into a SQL query:\n"
        f"{query}"
    )
    # Initialize your LLM (adjust the parameters as needed).
    llm = ChatOpenAI(temperature=0)
    response = llm(prompt)
    # Return the generated SQL as part of the updated state.
    return {"sql": response}

# Prepare an initial state containing your query and your table schema.
initial_state = {
    "query": "Show me all active users.",
    "schema": "users(id INTEGER, name TEXT, status TEXT)"
}

# Build the graph using StateGraph (this is the public API in LangGraph).
graph_builder = StateGraph(dict)
graph_builder.add_node("generate_sql", generate_sql_node)
graph_builder.add_edge(START, "generate_sql")
graph_builder.add_edge("generate_sql", END)

# Compile the graph (using an in-memory checkpointer for persistence).
graph = graph_builder.compile(checkpointer=MemorySaver())

# Invoke the graph with the initial state.
result = graph.invoke(initial_state)
print("Generated SQL query:")
print(result["sql"])


In [ ]:
# Ensure your file is not named "langgraph.py" to avoid shadowing the package.
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage


# This node uses your custom LLM to convert a natural language query into a SQL query.
def generate_sql_node(state):
    # The state already contains a "schema" field that is baked in.
    query = state["query"]
    schema = state["schema"]
    prompt = (
        "You are a SQL expert. Given the following table schema:\n"
        f"{schema}\n"
        "Convert the natural language query into a SQL query:\n"
        f"{query}"
    )
    # Use your custom LLM function to generate the SQL query.
    response = LLM(prompt)
    # Return the generated SQL as part of the updated state.
    return {"sql": response}

# Prepare an initial state containing your query and your table schema.
initial_state = {
    "query": "Show me all active users.",
    "schema": "users(id INTEGER, name TEXT, status TEXT)"
}

# Build the graph using the public StateGraph API.
graph_builder = StateGraph(dict)
graph_builder.add_node("generate_sql", generate_sql_node)
graph_builder.add_edge(START, "generate_sql")
graph_builder.add_edge("generate_sql", END)

# Compile the graph using an in-memory checkpointer for persistence.
graph = graph_builder.compile(checkpointer=MemorySaver())

# Invoke the graph with the initial state.
result = graph.invoke(initial_state)
print("Generated SQL query:")
print(result["sql"])


In [2]:
import pandas as pd

In [3]:
df2 = pd.read_csv("deepeval_results.csv")

In [4]:
df2.head()

,question,ground_truth,generated_sql,result
0,Find total contributions by each contributor.,"SELECT \r\n contribid,\r\n contrib,\r\n ...","```sql\nSELECT contrib, SUM(amount) AS total_c...",PASS
1,List all contributions made in the year 2022.,"SELECT \r\n contrib,\r\n recipient,\r\n ...",```sql\nSELECT * \nFROM campaign_contributions...,PASS
2,Find the top 5 recipients who received the hig...,"SELECT \r\n recipid,\r\n recipient,\r\n ...","```sql\nSELECT recipient, SUM(amount) as total...",PASS
3,Count the number of contributions made by cont...,SELECT \r\n COUNT(*) as total_contributions...,```sql\nSELECT COUNT(*) \nFROM campaign_contri...,PASS
4,Find the average contribution amount.,"SELECT \r\n ROUND(AVG(amount), 2) as averag...",```sql\nSELECT AVG(amount) FROM campaign_contr...,PASS
